In [118]:
# Day 16
import numpy as np

def parse_rules(rules_list):
    rules_dict = dict()
    for rule in rules_list:
        field, nums = rule.split(': ')
        nums = nums.replace(' or ', '-')
        nums = list(map(int, nums.split('-')))
        nums = list(range(nums[0], nums[1]+1)) + list(range(nums[2], nums[3]+1))
        rules_dict[field] = set(nums)
    return rules_dict

def invalidate_tickets(nearby_tickets, rules):
    invalid = list()
    valid_nums = set([num for sublist in rules.values() for num in sublist])
    rows_to_del = list()
    for r in range(nearby_tickets.shape[0]):
        for v in nearby_tickets[r]:
            if v not in valid_nums:
                invalid.append(v)
                rows_to_del.append(r)
    nearby_tickets = np.delete(nearby_tickets, rows_to_del, axis=0)
    return nearby_tickets, invalid

def get_this_field(col_num, ticket_matrix, fields_to_check, rules):
    for x in ticket_matrix[:,col_num]:
        for f in sorted(fields_to_check):
            if x not in rules[f]:
                fields_to_check.remove(f)        
        if len(fields_to_check) == 1:
            return next(iter(fields_to_check))

def get_field_per_col(rules, nearby_tickets):
    field_col_map = dict()
    cols_to_assign = set(range(nearby_tickets.shape[1]))
    possible_fields = set(rules.keys())
    while len(cols_to_assign) > 0:
        for c in sorted(cols_to_assign):
            fields_to_check = possible_fields.copy()
            this_field = get_this_field(c, nearby_tickets, fields_to_check, rules)
            if this_field != None:
                field_col_map[this_field] = c
                possible_fields.remove(this_field)
                cols_to_assign.remove(c)
    return field_col_map

def run(my_file):
    data = np.array([x.strip() for x in open(my_file)])
    breaks = np.where(data=='')[0]
    rules = parse_rules(data[:breaks[0]])
    your_ticket = data[breaks[0]+2]
    your_ticket = list(map(int, your_ticket.split(',')))
    nearby_tickets = data[breaks[1]+2:]
    nearby_tickets = np.array([list(map(int, t.split(','))) for t in nearby_tickets])
    
    nearby_tickets, invalid = invalidate_tickets(nearby_tickets, rules) #pt1
    field_col_map = get_field_per_col(rules, nearby_tickets) #pt2
#     print(field_col_map)    
    pt1 = sum(invalid)
    pt2 = 1
    for fld in field_col_map:
        if 'departure' in fld:
            pt2 *= your_ticket[field_col_map[fld]]
    print(pt1, pt2)

# my_file = 'test.txt'
my_file = 'day_16.txt'
run(my_file)

23925 964373157673
Wall time: 96 ms


In [294]:
# Day 17, CLEAN
import numpy as np
from scipy.signal import convolve

def get_active_cells(myfile, dims=3, cycles=6):
    data = [[c == '#' for c in line.strip()] for line in open(my_file)]
    grid = np.array(data).astype(int)
#     print(grid)
    grid = np.expand_dims(grid, axis=tuple(range(dims-2))) #insert a dim of length one for each dim past 2
    
    #kernel is a generic object length 3 in every dim, i.e. a cube of (3*3*3) surrounding and including active cell in 3D
    kernel = np.ones((3, )*dims, dtype=int) 
    kernel[(1, )*dims] = 0 #zero center of kernel, i.e. don't count active cell; since kernel is len 3 center is 1,1,...
    
    for _ in range(cycles): #repeat these tasks as many times as asked for
        grid = np.pad(grid, pad_width=1, mode='constant', constant_values=0) #pad grid with zeros
        #center kernel (hollowed cube of ones) around each cell in the grid, respectively, and multiply where overlap
        neighbors = convolve(grid, kernel, mode='same') #output is same size as 'grid', value of each cell is # neighbors
        #create bool grids indicating where values need to be changed
        set_inactive = np.logical_and(grid == 1, np.floor_divide(neighbors, 2) != 1)
        set_active = np.logical_and(grid == 0, neighbors == 3)
        #now change those values
        grid[set_inactive] = 0
        grid[set_active] = 1
    print(grid.sum())
    
# my_file = 'test.txt'
my_file = 'day_17.txt'
get_active_cells(myfile, dims=3)
get_active_cells(myfile, dims=4)

388
2280


In [245]:
# Day 17, Part One, ORIGINAL
import numpy as np
# my_file = 'test.txt'
my_file = 'day_17.txt'

data = [[c == '#' for c in line.strip()] for line in open(my_file)]
grid = np.array(data).astype(int)

grid = grid[:,:,np.newaxis] #convert grid from shape (x,y,0) to (x,y,1)
new_shape = np.array(grid.shape) + 2 #provide an extra buffer to avoid boundary checks
for n in range(6):
    new_shape += 2 #grid can *at most* expand by 2 in each direction per iter
    new_grid = np.zeros(new_shape)
    s = 2 if n == 0 else 1 #layers of 'slack'
    new_grid[s:grid.shape[0]+s, s:grid.shape[1]+s, s:grid.shape[2]+s] = grid.copy()
    next_grid = new_grid.copy()
    for x in range(1,new_grid.shape[0]-1):
        for y in range(1,new_grid.shape[1]-1):
            for z in range(1,new_grid.shape[2]-1):
                is_active = new_grid[x, y , z] #is this cube currently set to active?
                cube_to_check = new_grid[x-1:x+2, y-1:y+2, z-1:z+2]
                active_neighbors = int(cube_to_check.sum()) - is_active #don't count yourself!
                if is_active:
                    if active_neighbors != 2 and active_neighbors != 3:
                        next_grid[x, y, z] = 0 #this cell becomes inactive
                elif active_neighbors == 3:
                    next_grid[x, y, z] = 1 #this cell becomes active (prev inactive)
    grid = next_grid.copy()
#         print(n, int(grid.sum()))
pt1 = grid.sum().astype(int)
print(f'pt1: {pt1}')

pt1: 388


In [292]:
# Day 17, Part Two ~840ms
import numpy as np
# my_file = 'test.txt'
my_file = 'day_17.txt'

data = [[c == '#' for c in line.strip()] for line in open(my_file)]
grid = np.array(data).astype(int)

grid = grid[:,:,np.newaxis] #convert grid from shape (x,y,0) to (x,y,1)
grid = grid[:,:,:,np.newaxis] #convert grid from shape (x,y,1,0) to (x,y,1,1)
new_shape = np.array(grid.shape) + 2 #provide an extra buffer to avoid boundary checks
for n in range(6):
    new_shape += 2 #grid can *at most* expand by 2 in each direction per iter
    new_grid = np.zeros(new_shape)
    s = 2 if n == 0 else 1 #layers of 'slack'
    new_grid[s:grid.shape[0]+s, s:grid.shape[1]+s, s:grid.shape[2]+s, s:grid.shape[3]+s] = grid.copy()
    next_grid = new_grid.copy()
    for x in range(1,new_grid.shape[0]-1):
        for y in range(1,new_grid.shape[1]-1):
            for z in range(1,new_grid.shape[2]-1):
                for w in range(1,new_grid.shape[3]-1):
                    is_active = new_grid[x, y, z, w] #is this cube currently set to active?
                    cube_to_check = new_grid[x-1:x+2, y-1:y+2, z-1:z+2, w-1:w+2]
                    active_neighbors = int(cube_to_check.sum()) - is_active #don't count yourself!
                    if is_active:
                        if active_neighbors != 2 and active_neighbors != 3:
                            next_grid[x, y, z, w] = 0 #this cell becomes inactive
                    elif active_neighbors == 3:
                        next_grid[x, y, z, w] = 1 #this cell becomes active (prev inactive)
    grid = next_grid.copy()
#         print(n, int(grid.sum()))
pt2 = grid.sum().astype(int)
print(f'pt2: {pt2}')

pt2: 2280
Wall time: 843 ms


In [397]:
#Day 18
import numpy as np

def clean_line(line):
    line = line.replace('(', '( ')
    line = line.replace(')', ' )')
    return line.split(' ')

def eval_chunk(chunk):
    while len(chunk) != 1:
        res = eval(''.join(chunk[0:3]))
        del chunk[0:3]
        chunk.insert(0, str(res))
    return chunk[0]

def eval_chunk2(chunk):
    while len(chunk) != 1:
        if '+' in chunk:
            i = chunk.index('+')
        elif '*' in chunk:
            i = chunk.index('*')
        res = eval(''.join(chunk[i-1:i+2]))
        del chunk[i-1:i+2]
        chunk.insert(i-1, str(res))
    return chunk[0]

def simplify_all_parens(chars1):
    chars2 = chars1.copy()
    while ')' in chars1:
        cp_idx = chars1.index(')') #find first closed paren
        len_chunk = chars1[cp_idx::-1].index('(')
        op_idx = cp_idx - len_chunk
        chunk_val1 = eval_chunk(chars1[op_idx+1:cp_idx])
        chunk_val2 = eval_chunk2(chars2[op_idx+1:cp_idx])
        del chars1[op_idx:cp_idx+1]
        del chars2[op_idx:cp_idx+1]
        chars1.insert(op_idx, chunk_val1)
        chars2.insert(op_idx, chunk_val2)
    return chars1, chars2

def run(my_file):
    data = [x.strip() for x in open(my_file)]
    results_by_line = np.array([[],[]], dtype=int) #a (truly) empty array shape 2,1
    for line in data:
        #get a list of all nums/operators, incl parens
        all_chars = clean_line(line)
        #simplify away all the parens
        chars1, chars2 = simplify_all_parens(all_chars)
        #eval what remains
        new_res = np.array([[int(eval_chunk(chars1))], [int(eval_chunk2(chars2))]]) #pt1, pt2, in shape 2,1
        results_by_line = np.hstack((results_by_line, new_res)) #tack on to our running totals
    return results_by_line[0,:].sum(), results_by_line[1,:].sum()

# my_file = 'test.txt'
my_file = 'day_18.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

pt1: 4491283311856, pt2: 68852578641904


In [530]:
#Day 19
def initialize(data):
    rules = dict() #r_num (str) --> set of all letter permutations which satisfy
    dependents = dict() #r_num (str) --> set of all other rules which need to be defined first
    sequences = dict() #r_num (str) --> list of possible orderings of other rules (could be one or two)
    messages = list()    
    for line in data:
        if line != '' and line[0].isnumeric():
            r_num, seqs = line.split(': ')
            if seqs[0] == '"':
                rules[r_num] = {seqs.strip('"')}
            else:
                if '|' in seqs:
                    seqs = seqs.split(' | ')
                    for i in range(len(seqs)):
                        seqs[i] = seqs[i].split(' ')
                else:
                    seqs = [seqs.split(' ')]
                dependents[r_num] = set()
                sequences[r_num] = list()
                for s in seqs:
                    sequences[r_num].append(s)
                    for r in s:
                        dependents[r_num].add(r)
        elif line != '':
            messages.append(line)
    return rules, dependents, sequences, messages

def valid_for_pt2(msg, rule42, rule31, len42):
    #new rules for pt2: {'8':'42 | 42 8', '11':'42 31 | 42 11 31'}
    #this means that additionally valid strings would:
    #(1) begin with any # of consecutive chunks which are valid for rule42
    #(2) end with some # of consecutive rule31 chunks, but this must be at least one fewer in length than the 42s @ start
    chunks = [msg[i:i+len42] for i in range(0, len(msg), len42)]
    chunk_rules = [(c in rule42)*42 + (c in rule31)*31 for c in chunks]
    if chunk_rules[0] == 42 and 31 in chunk_rules and 0 not in chunk_rules:
        if chunk_rules[chunk_rules.index(31):].count(42) == 0 and chunk_rules.count(31) < chunk_rules.count(42):
            return True
    return False

def run(my_file):
    data = [x.strip() for x in open(my_file)]
    #interpret data, create the dictionaries we will use to trace rules
    rules, dependents, sequences, messages = initialize(data)
    #solve any rules which already have all dependents, repeat until no dependents remain
    while dependents != dict():
        for r_num in sorted(dependents):
            if all([d in rules for d in dependents[r_num]]):
                rules[r_num] = set()
                for seq in sequences[r_num]:
                    poss_strs = {0:list(), 1:list(), 2:list()}
                    if len(seq) < 3:
                        poss_strs[2] = ['']
                    if len(seq) < 2:
                        poss_strs[1] = ['']
                    for i in range(len(seq)):
                        for str_ in rules[seq[i]]:
                            poss_strs[i].append(str_)
                    for str0 in poss_strs[0]:
                        for str1 in poss_strs[1]:
                            for str2 in poss_strs[2]:
                                rules[r_num].add(str0+str1+str2)
                dependents.pop(r_num, None)
    rule42 = rules['42'] #need these for pt2
    rule31 = rules['31']
    len42 = len(next(iter(rule42))) #assume rules 42/31 same len
    pt1, pt2 = 0, 0
    for msg in messages:
        if msg in rules['0']:
            pt1 += 1   
        elif valid_for_pt2(msg, rule42, rule31, len42):
            pt2 += 1
    return pt1, pt1+pt2

# my_file = 'test.txt'
my_file = 'day_19.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {pt1}, pt2: {pt2}')

pt1: 111, pt2: 343


In [727]:
# Day 20 --Jurrassic Jigsaw--
import numpy as np
from scipy.signal import convolve
    
def get_match(edge, all_edges, tile_id):
    #check if there is a match for this edge(besides itself), or for the reverse of this edge
    rev_edge = edge[::-1]
    if edge.tobytes() in all_edges:
        for t_id in all_edges[edge.tobytes()]:
            if t_id != tile_id:
                return t_id
    if rev_edge.tobytes() in all_edges:
        for t_id in all_edges[rev_edge.tobytes()]:
            if t_id != tile_id:
                return t_id

def get_dicts(tiles):
    all_edges = dict()
    id_edges_map = dict()
    id_grid_map = dict()
    for tile in tiles:
        if tile == '': #EoF condition
            continue
        lines = tile.split('\n')
        grid = [[c == '#' for c in line] for line in lines[1:]]
        grid = np.array(grid).astype(int)
        tile_id = int(lines[0].strip('Tile ').strip(':'))
        edges = [grid[0,:], grid[-1,:], grid[:,0], grid[:,-1]]
        id_edges_map[tile_id] = edges
        id_grid_map[tile_id] = grid
        for edge in edges:
            if edge.tobytes() in all_edges:
                all_edges[edge.tobytes()].append(tile_id)
            else:
                all_edges[edge.tobytes()] = [tile_id]
    return all_edges, id_edges_map, id_grid_map

def get_corners(id_edges_map, all_edges):
    corners = list()
    for tile_id in id_edges_map:
        matched_edges = 0
        for edge in id_edges_map[tile_id]:
            if get_match(edge, all_edges, tile_id) != None: 
                matched_edges += 1
        if matched_edges == 2:
            corners.append(tile_id)
    return corners

def create_supergrid(id_grid_map, all_edges, seed):
    side_len = np.sqrt(len(id_grid_map)).astype(int)
    seed_grid = id_grid_map[seed].copy()
    #force matches to occur on bottom, right
    if get_match(seed_grid[0,:], all_edges, seed) is not None:
        seed_grid = np.flip(seed_grid, 0)
    if get_match(seed_grid[:,0], all_edges, seed) is not None:
        seed_grid = np.flip(seed_grid, 1)
#     print('first seed:', seed)
    row_seeds = [seed]
    row_grids = {seed:seed_grid}
    for _ in range(side_len-1):
        curr_bottom = seed_grid[-1,:]
        next_seed = get_match(curr_bottom, all_edges, seed)
#         print('next seed:', next_seed)
        next_seed_grid = id_grid_map[next_seed].copy()
        next_top = next_seed_grid[0,:]
        while not np.array_equal(curr_bottom, next_top):
            if np.array_equal(curr_bottom, next_top[::-1]):
                next_seed_grid = np.flip(next_seed_grid, 1)
            else:
                next_seed_grid = np.rot90(next_seed_grid)
            next_top = next_seed_grid[0,:]
        row_seeds.append(next_seed)
        row_grids[next_seed] = next_seed_grid
        seed = next_seed
        seed_grid = next_seed_grid.copy()
    rows = []
    for this_tile in row_seeds:
#         print('-START ROW-', this_tile)
        row = row_grids[this_tile]
        for _ in range(side_len-1):
            curr_rt = row[:,-1]
            next_tile = get_match(curr_rt, all_edges, this_tile)
            this_tile = next_tile
#             print('next tile:', next_tile)
            next_tile_grid = id_grid_map[next_tile].copy()
            next_left = next_tile_grid[:,0]
            while not np.array_equal(curr_rt, next_left):
                if np.array_equal(curr_rt, next_left[::-1]):
                    next_tile_grid = np.flip(next_tile_grid, 0)
                else:
                    next_tile_grid = np.rot90(next_tile_grid)
                next_left = next_tile_grid[:,0]
            row = np.hstack((row, next_tile_grid))
        rows.append(row)
    supergrid = rows[0]
    for row in rows[1:]:
        supergrid = np.vstack((supergrid,row))
    idx_to_del = list(range(0,supergrid.shape[0],10))+list(range(9,supergrid.shape[0],10))
    supergrid = np.delete(supergrid, idx_to_del, axis=0)
    supergrid = np.delete(supergrid, idx_to_del, axis=1)
    return supergrid

def find_monsters(supergrid):
    monster = np.zeros((3,20)).astype(int)
    monster[0,18] = 1
    monster[1,[0,5,6,11,12,17,18,19]] = 1
    monster[2,[1,4,7,10,13,16]] = 1
    overlap = convolve(supergrid, monster, mode='same')
    num_monsters = np.sum(overlap == monster.sum())
    iters = 0
    while num_monsters == 0:
        iters += 1
        supergrid = np.rot90(supergrid)
        if iters % 4 == 0: #every fourth go, rotate back to original, then flip
            supergrid = np.flip(supergrid, 1)
        overlap = convolve(supergrid, monster, mode='same')
        num_monsters = np.sum(overlap == monster.sum())
#     print(iters, num_monsters)
    return supergrid.sum() - num_monsters*monster.sum()

def run(my_file):
    tiles = open(my_file).read().split('\n\n')
    #get(1)dict of unique edge -> [ids w/that edge] (2)map of each tile_id to its four edges(as arrays) (3)all grids
    all_edges, id_edges_map, id_grid_map = get_dicts(tiles) 
    #find tiles that only have two edges which match another edge, call these corners
    corners = get_corners(id_edges_map, all_edges)
    supergrid = create_supergrid(id_grid_map, all_edges, corners[0])
#     print(supergrid.sum())
    pt2 = find_monsters(supergrid) #sum number of '#' in grid which are *not* part of a monster
    return np.prod(np.array(corners).astype(float)), pt2
    
# my_file = 'test.txt'
my_file = 'day_20.txt'
pt1, pt2 = run(my_file)
print(f'pt1: {int(pt1)}, pt2: {pt2}')

pt1: 16192267830719, pt2: 1909
